In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### 1.1 Configure your Gemini API Key


In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
try: 
    google_api_key = os.getenv('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = google_api_key
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")


✅ Gemini API key setup complete.


### 1.2 Import ADK components


In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [19]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings", # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [20]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [21]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")

✅ root_agent created.


In [22]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements in quantum computing and what do they mean for AI?")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/soroushav/Desktop/kaggle/5_day_agenticai_intensive_course/venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/soroushav/Desktop/kaggle/5_day_agenticai_intensive_course/venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/soroushav/Desktop/kaggle/5_day_agenticai_intensive_course/venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


ResearchCoordinator > Here is a concise summary of the latest advancements in quantum computing and their implications for AI:

*   **Accelerated AI Training and Optimization:** Quantum computers can drastically speed up AI model training and fine-tuning by performing complex calculations faster, particularly for deep learning.
*   **Improved Data Processing and Problem Solving:** Quantum machine learning offers more efficient ways to process large datasets for better predictions and enables tackling complex problems (like drug discovery) beyond classical computer capabilities.
*   **Increased Efficiency and Hardware Innovations:** Quantum AI has the potential for greater energy efficiency, and ongoing advancements in quantum hardware (like improved qubits and error correction) are crucial for realizing its full potential.
*   **Hybrid Systems and Future Development:** Near-term AI applications will likely leverage hybrid quantum-classical systems, and significant progress in quantum e

## 🚥 Sequential Workflows - The Assembly Line


<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

### Example: Blog Post Creation with Sequential Agents

Let's build a system with three specialized agents:

1. **Outline Agent** - Creates a blog outline for a given topic
2. **Writer Agent** - Writes a blog post
3. **Editor Agent** - Edits a blog post draft for clarity and structure

In [23]:
outline_agent = Agent(
    name='OutlineAgent',
    model='gemini-2.5-flash-lite',
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",
)
print("✅ outline_agent created.")

✅ outline_agent created.


In [24]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft", # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [25]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog", # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [26]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [27]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the benefits of multi-agent systems for software developers")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/soroushav/Desktop/kaggle/5_day_agenticai_intensive_course/venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ## Outline:

**Headline:** Unleash the Power of Collaboration: How Multi-Agent Systems Will Revolutionize Your Software Development

**Introduction Hook:** Tired of monolithic codebases and siloed development teams? Imagine a world where your software components aren't just lines of code, but intelligent, autonomous agents working together seamlessly. That's the promise of Multi-Agent Systems (MAS), and it's time for software developers to harness their transformative potential.

---

### Section 1: Breaking Down Complexity with Decentralized Intelligence

*   **Divide and Conquer:** MAS allows you to decompose complex problems into smaller, more manageable sub-problems, each handled by an individual agent. This significantly reduces cognitive load for developers and simplifies debugging.
*   **Enhanced Modularity and Reusability:** Each

## 🛣️ Parallel Workflows - Independent Researchers


<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

### Example: Parallel Multi-Topic Research

Let's build a system with four agents:

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

In [28]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research", # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [29]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research", # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [30]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research", # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [31]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary", # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [32]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [34]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/soroushav/Desktop/kaggle/5_day_agenticai_intensive_course/venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > ## Daily Executive Briefing: November 10, 2025

**Tech:**
Artificial Intelligence (AI) continues its rapid evolution, with agentic AI systems capable of complex, autonomous tasks gaining traction. This necessitates shifts in infrastructure orchestration to handle demanding AI workloads. Emerging technologies like 5G expansion, quantum computing, and green energy remain key areas of innovation, shaping diverse industries.

**Health:**
The health sector is embracing personalized approaches, driven by AI and data analytics. Genomics and gene editing are advancing towards clinical applications for genetic conditions. Technological solutions for mental wellness, including VR/AR sessions and supportive chatbots, are emerging to improve accessibility. Consumers are increasingly focused on gut health, functional foods, and healthy aging.

**Finance:**
Key financ

## ➰ Loop Workflows - The Refinement Cycle


<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

### Example: Iterative Story Refinement

Let's build a system with two agents:

1. **Writer Agent** - Writes a draft of a short story
2. **Critic Agent** - Reviews and critiques the short story to suggest improvements

In [43]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story", # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [44]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique", # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [45]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

✅ exit_loop function created.


In [46]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    
    output_key="current_story", # It overwrites the story with the new, refined version.
    tools=[FunctionTool(exit_loop)], # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


In [47]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=5, # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [48]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/soroushav/Desktop/kaggle/5_day_agenticai_intensive_course/venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias traced the damp, salt-crusted lines on the parchment. It had washed ashore, snagged on the rocks below the lighthouse, a beacon of impossible light. Not the steady, reassuring beam he knew, but a pulsating, otherworldly glow emanating from within the map itself. The ink, if it was ink, shifted and reformed, forming constellations he’d never seen, coastlines that defied geography. He held it close, the light warming his calloused hands. A strange hum vibrated through the tower, a song of distant stars and forgotten seas. For the first time in fifty years, the solitude of his lighthouse felt less like a comfort and more like a cage. The map whispered of adventure, of places no human foot had ever trod.
CriticAgent > This is a compelling start to a story, brimming with atmosphere and intrigue. Here's a breakdown:

**Plot:** Th

CriticAgent > The story is well-written and complete. The plot is engaging, the character of Elias is well-established, and the pacing effectively builds intrigue. The descriptions are vivid and create a strong sense of atmosphere.

APPROVED


CriticAgent > APPROVED


CriticAgent > APPROVED


### Section 6
## Summary - Choosing the Right Pattern

### Decision Tree: Which Workflow Pattern?


<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (sub_agents)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |